# CMP STEP 3: Measuring Item Bias

In [1]:
import numpy as np
import pandas as pd
import random

import os
import json
from tqdm import tqdm

from scipy.spatial.distance import cosine as cosdist
import scipy.stats as st

import matplotlib.pyplot as plt

In [2]:
EMBEDDINGS = 'data/'
BIAS = 'bias/'
MODE = 'masked'  # change mode to either 'normal' or 'masked'

### Load vignette & strategy embeddings

In [3]:
# load pre-encoded vignette embeddings
vignette_embeddings = np.load(EMBEDDINGS + 'vignette_embeddings.npy')

In [4]:
# load pre-encoded strategy embeddings
if MODE == 'normal':
    strategy_embeddings = np.load(EMBEDDINGS + 'strategy_embeddings.npy')
elif MODE == 'masked':
    strategy_embeddings = np.load(EMBEDDINGS + 'strategy_embeddings_masked.npy')

### WEAT (Caliskan et al., 2017) adapted one-vs-many

In [5]:
strategies_per_item = [0, 679, 680, 725, 718, 689, 671, 571, 650, 663]

In [6]:
def WEAT_one_vs_all(item_no=1, random_seed=15):
    """calculates the WEAT statistic for bias dimension item_no-all_other_items:
       Are answers to item_no closer to vignette of item_no in the embedding space
       compared to the vignettes of all other items (averaged)?"""    
    start = np.sum(strategies_per_item[:item_no])
    stop = start + strategies_per_item[item_no]
    other_items = np.delete(strategy_embeddings, range(start, stop), axis=0)
    
    X = strategy_embeddings[start:stop]
    Y = other_items

    A = vignette_embeddings[item_no - 1]
    B = np.delete(vignette_embeddings, (item_no - 1), axis=0)
    
    X_boot = random.choices(X, k=len(X))
    Y_boot = random.choices(Y, k=len(X)) 
    
    s_WAB = []
    
    s_XAB = 0
    for sentence in X_boot:
        inner_item_dist = cosdist(sentence, A)
        cross_item_dist = np.mean([cosdist(sentence, B[x]) for x in range(len(B))])
        s_wAB = inner_item_dist - cross_item_dist
        s_WAB.append(s_wAB)
        s_XAB += s_wAB
    mean_s_XAB = s_XAB / len(X_boot)

    s_YAB = 0
    for sentence in Y_boot:
        cross_item_dist = cosdist(sentence, A)
        inner_item_dist = np.mean([cosdist(sentence, B[x]) for x in range(len(B))])
        s_wAB = cross_item_dist - inner_item_dist
        s_WAB.append(s_wAB)
        s_YAB += s_wAB
    mean_s_YAB = s_YAB / len(Y_boot)

    s_XYAB = s_XAB - s_YAB
    
    effect_size = (mean_s_XAB - mean_s_YAB) / np.std(s_WAB)

    return s_XYAB, effect_size

### Calculate item bias statistics & effect sizes, save and display

In [7]:
# calculate WEAT statistics for all items: includes (bootstrapped bias statistic, bootstrapped effect size)
# save to dictionary/json

# bias_results = {}

# for item in range(1, 10):
    # compute statistic for 1000 times with different seeds to produce confidence interval and estimate mean
#     print(f'Bootstrapping for item {item}...')
#     bias_boot = []
#     es_boot = []

#     for seed in tqdm(np.arange(1000)):
#         WEAT = WEAT_one_vs_all(item_no=item, random_seed=seed)
#         bias_boot.append(WEAT[0])
#         es_boot.append(WEAT[1])
    
#     bias_results[item] = (bias_boot, es_boot)

In [8]:
# save bias results to file
# with open(BIAS + f'bias_boot_{MODE}.json', 'w') as f:
    # f.write(json.dumps(bias_results))

In [9]:
# reload pre-computed bias results
with open(BIAS + f'bias_boot_{MODE}.json', 'r') as f:
    bias_results = json.loads(f.read())

In [10]:
# display bias results in table
df_one_vs_all = pd.DataFrame(columns=['statistic (mean)', 'CI (95%)'], index= range(1, 10))

for item in range(1, 10):
    mu, sigma = st.norm.fit(bias_results[str(item)][0])
    lower_95 = mu + sigma * st.norm.ppf(0.025, loc=mu, scale=sigma)
    upper_95 = mu - sigma * st.norm.ppf(0.025, loc=mu, scale=sigma)
    
    df_one_vs_all.loc[item]['statistic (mean)'] = f'{mu:.2f}'
    df_one_vs_all.loc[item]['CI (95%)'] = f'[{lower_95:.2f}; {upper_95:.2f}]'

df_one_vs_all

,statistic (mean),CI (95%)
1,-0.65,[-0.78; -0.52]
2,-1.53,[-1.70; -1.36]
3,-2.40,[-2.89; -1.91]
4,-1.15,[-1.31; -1.00]
5,-1.99,[-2.24; -1.74]
6,-0.69,[-0.74; -0.63]
7,-0.68,[-0.77; -0.59]
8,-1.08,[-1.21; -0.95]
9,-1.49,[-1.75; -1.23]


In [11]:
# display effect size results in table
df_one_vs_all = pd.DataFrame(columns=['effect size (mean)', 'CI (95%)'], index= range(1, 10))

for item in range(1, 10):
    mu, sigma = st.norm.fit(bias_results[str(item)][1])
    lower_95 = mu + sigma * st.norm.ppf(0.025, loc=mu, scale=sigma)
    upper_95 = mu - sigma * st.norm.ppf(0.025, loc=mu, scale=sigma)
    
    df_one_vs_all.loc[item]['effect size (mean)'] = f'{mu:.2f}'
    df_one_vs_all.loc[item]['CI (95%)'] = f'[{lower_95:.2f}; {upper_95:.2f}]'

df_one_vs_all

,effect size (mean),CI (95%)
1,-0.25,[-0.26; -0.23]
2,-0.75,[-0.79; -0.71]
3,-0.66,[-0.69; -0.62]
4,-0.50,[-0.53; -0.47]
5,-0.85,[-0.89; -0.81]
6,-0.50,[-0.53; -0.47]
7,-0.36,[-0.38; -0.33]
8,-0.57,[-0.60; -0.53]
9,-0.53,[-0.56; -0.49]
